In [80]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft,dct
import librosa
from scipy import signal
import soundfile as sf
from librosa.display import specshow
import glob

In [139]:
# # separate 

carTrain = glob.glob("cars/train/*.wav")
carTest = glob.glob("cars/test/*.wav")

tramTrain = glob.glob("trams/train/*.wav")
tramTest = glob.glob("trams/test/*.wav")

dataset=[]
labels = []


In [140]:
def importFiles(files, label):
    dataset = []
    labels = []
    for file in files:
        data,sr = librosa.load(file)
        data = librosa.effects.trim(data, top_db=20, frame_length=1024, hop_length=512)[0] 
        # print(data.size)
        # print(data.shape)# Desired length in samples
        desired_length = sr * 5
        # # Initialize a new array of zeros with the desired length
        fixed_length_data = np.zeros(desired_length)
        #  Check the length of the original data
        original_length = len(data)
        # # If original data is longer than desired length, truncate it
        # # If it is shorter, pad with zeros
        if original_length > desired_length:
            fixed_length_data = data[:desired_length]
        else:
            fixed_length_data[:original_length] = data
        # # Now use fixed_length_data as your adjusted data
        data = fixed_length_data




        labels.append(label)
        dataset.append(data)

    return dataset,labels

In [141]:
# # separate

car_dataset, car_label  = importFiles(carTrain, 0)
car_test_dataset, car_test_label = importFiles(carTest, 0)
tram_dataset, tram_label = importFiles(tramTrain, 1)
tram_test_dataset, tram_test_label = importFiles(tramTest, 1)

In [142]:
tram_dataset_array = np.array(tram_dataset)
car_dataset_array = np.array(car_dataset)
tram_label_array = np.array(tram_label)
car_label_array = np.array(car_label)


In [143]:
print(tram_dataset_array.shape)
print(car_dataset_array.shape)
print(tram_label_array.shape)
print(car_label_array.shape)


(8, 110250)
(7, 110250)
(8,)
(7,)


In [91]:
# fs = 44000
# def extractFeatures(dataset):
#     features=[]
#     for audio in dataset:
#         mfccs = librosa.feature.mfcc(y=np.asarray(audio), sr=fs, n_mfcc=40)
#         features.append(mfccs)
    
#         # spectral spread
#         spectral_bandwidth = librosa.feature.spectral_bandwidth(y=np.asarray(audio), sr=fs)

#         # spectral energy
#         spectral_centroid = librosa.feature.spectral_centroid(y=np.asarray(audio), sr=fs)

#         # spectral density
#         spectral_contrast = librosa.feature.spectral_contrast(y=np.asarray(audio), sr=fs)

#         #  rate of sign-changes in the signal
#         zerocrossing_rate = librosa.feature.zero_crossing_rate(y=np.asarray(audio))

#         #  frequency below which a certain percentage of the power spectrum is concentrated
#         spectral_rolloff = librosa.feature.spectral_rolloff(y=np.asarray(audio), sr=fs)
#         combined_features = np.hstack([np.mean(mfccs, axis=1), np.std(mfccs, axis=1),
#                                        np.mean(spectral_bandwidth), np.std(spectral_bandwidth),
#                                        np.mean(spectral_centroid), np.std(spectral_centroid),
#                                        np.mean(spectral_contrast), np.std(spectral_contrast),
#                                        np.mean(zerocrossing_rate), np.std(zerocrossing_rate),
#                                        np.mean(spectral_rolloff), np.std(spectral_rolloff)])

#         # features list
#         features.append(combined_features)
#     return features

In [92]:
# car_dataset=extractFeatures(car_dataset_array)
# tram_dataset=extractFeatures(tram_dataset_array)

In [144]:
dataset = np.concatenate([tram_dataset, car_dataset], axis=0)
labels = np.concatenate([tram_label, car_label], axis=0)


In [145]:
print(dataset.shape)
print(labels.shape)

(15, 110250)
(15,)


In [146]:
fs = 44000
features=[]


for audio in dataset:
    mfccs = librosa.feature.mfcc(y=np.asarray(audio), sr=fs, n_mfcc=40)    
    # spectral spread
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=np.asarray(audio), sr=fs)

    # spectral energy
    spectral_centroid = librosa.feature.spectral_centroid(y=np.asarray(audio), sr=fs)

    # spectral density
    spectral_contrast = librosa.feature.spectral_contrast(y=np.asarray(audio), sr=fs)

    #  rate of sign-changes in the signal
    zerocrossing_rate = librosa.feature.zero_crossing_rate(y=np.asarray(audio))

    #  frequency below which a certain percentage of the power spectrum is concentrated
    spectral_rolloff = librosa.feature.spectral_rolloff(y=np.asarray(audio), sr=fs)
    combined_features = np.hstack([np.mean(mfccs, axis=1), np.std(mfccs, axis=1),
                                   np.mean(spectral_bandwidth), np.std(spectral_bandwidth),
                                   np.mean(spectral_centroid), np.std(spectral_centroid),
                                   np.mean(spectral_contrast), np.std(spectral_contrast),
                                   np.mean(zerocrossing_rate), np.std(zerocrossing_rate),
                                   np.mean(spectral_rolloff), np.std(spectral_rolloff)])

    # features list
    features.append(combined_features)
    #features.append(features)


In [155]:
print(len(features))

15


In [164]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [119]:
# def flatten_features(features):
#     flattened_features = []
#     for feature in features:
#         flattened = np.hstack([np.mean(feature, axis=1), np.std(feature, axis=1)])
#         flattened_features.append(flattened)
#     return flattened_features


# flattened_features = flatten_features(features)

# scaler = StandardScaler()
# flattened_features = scaler.fit_transform(features)

In [ ]:
# # separate 


In [165]:
# # all
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.6666666666666666
